In [1]:
import pandas as pd
import numpy as np
from psql_etl import establish_connection

In [2]:
db_con = establish_connection()
customers = pd.read_sql('SELECT * FROM customers;', db_con)

In [3]:
customers.head()

,id,signup_date,loyalty,location,gender,age,favorite_movie_line,number_of_purchases,value_of_purchases,total_standard_points,total_points_redeemed
0,19,2012-01-01,control,SF,None,wise,None,NaN,NaN,NaN,NaN
1,30,2012-01-01,control,LA,None,None,"Frankly, my dear, I don't give a damn.",NaN,NaN,NaN,NaN
2,35,2012-01-02,control,None,A,None,"Frankly, my dear, I don't give a damn.",NaN,NaN,NaN,NaN
3,51,2012-01-02,control,LA,B,wise,"Frankly, my dear, I don't give a damn.",NaN,NaN,NaN,NaN
4,66,2012-01-03,control,SF,None,wise,"Frankly, my dear, I don't give a damn.",NaN,NaN,NaN,NaN


## Data Coverage

In [4]:
## data coverage
customers.count() / customers.count()['id']

id                       1.00000
signup_date              1.00000
loyalty                  1.00000
location                 0.49895
gender                   0.50130
age                      0.51010
favorite_movie_line      0.49815
number_of_purchases      0.76700
value_of_purchases       0.76700
total_standard_points    0.76700
total_points_redeemed    0.76700
dtype: float64

In [5]:
customers.notnull().groupby(['gender', 'location']).count()['id'].unstack(1)

location,False,True
gender,,
False,4984,4990
True,5037,4989


In [6]:
customers.notnull().groupby(['gender', 'age']).count()['id'].unstack(1)

age,False,True
gender,,
False,4941,5033
True,4857,5169


In [7]:
customers.notnull().groupby(['gender', 'favorite_movie_line']).count()['id'].unstack(1)

favorite_movie_line,False,True
gender,,
False,4983,4991
True,5054,4972


## Classifying a variable given a list of input features

In [8]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

def fit_transform_split_data(input_features, target_feature, df):
    cols = input_features + [target_feature]
    df_base = df.loc[:, df.columns.isin(cols)].dropna().copy()
    print(df_base.shape[0])
    le = LabelEncoder()
    for feature in input_features:
        df_base[feature] = le.fit_transform(df_base[feature])
    df_base[target_feature] = le.fit_transform(df_base[target_feature])
    X_train, X_test, y_train, y_test = train_test_split(df_base.loc[:, df_base.columns.isin(input_features)],
                                                        df_base.loc[:, target_feature],
                                                        random_state=1)
    
    return le, X_train, X_test, y_train, y_test

In [9]:
le, X_train, X_test, y_train, y_test = fit_transform_split_data(
    ['gender', 'age', 'favorite_movie_line', 'number_of_purchases', 'value_of_purchases', 'total_points_redeemed'], 
    'location',
    customers)

973


In [10]:
from sklearn.metrics import precision_score
from sklearn.metrics import classification_report

In [11]:
from sklearn.tree import DecisionTreeClassifier 

tree_model = DecisionTreeClassifier().fit(X_train, y_train)
y_pred = tree_model.predict(X_test)
print(precision_score(y_test, y_pred, average='micro'))
print(classification_report(y_test, y_pred))

0.418032786885
             precision    recall  f1-score   support

          0       0.39      0.35      0.37        78
          1       0.46      0.45      0.46        99
          2       0.39      0.45      0.42        67

avg / total       0.42      0.42      0.42       244



In [12]:
from sklearn import svm

svm_model = svm.SVC(decision_function_shape='ovo', random_state=1).fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.67      0.03      0.05        78
          1       0.41      0.96      0.57        99
          2       0.57      0.06      0.11        67

avg / total       0.53      0.41      0.28       244



In [13]:
from sklearn.naive_bayes import GaussianNB

gnb = GaussianNB().fit(X_train, y_train)
y_pred = gnb.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.40      0.05      0.09        78
          1       0.35      0.18      0.24        99
          2       0.31      0.84      0.45        67

avg / total       0.35      0.32      0.25       244



In [14]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=5, random_state=1).fit(X_train, y_train)
y_pred = rfc.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.41      0.40      0.41        78
          1       0.46      0.51      0.48        99
          2       0.40      0.36      0.38        67

avg / total       0.43      0.43      0.43       244



## Building manual classifications

In [15]:
customers.groupby('age').mean()

,id,number_of_purchases,value_of_purchases,total_standard_points,total_points_redeemed
age,,,,,
oldish,9993.440718,11.845868,34181.374188,827.669452,1267.409471
wise,10071.716967,5.031768,7017.213398,372.044199,44.889503
young,10120.424449,10.060530,14247.704852,518.024012,317.658829
youthful,10006.898894,13.256585,55971.872389,1114.173479,2402.361490


In [16]:
customers.groupby('gender').mean()

,id,number_of_purchases,value_of_purchases,total_standard_points,total_points_redeemed
gender,,,,,
A,9828.236189,9.315818,15626.363985,497.958402,418.719212
B,10048.803642,10.254912,25624.148699,688.151885,865.639936
C,10127.891401,9.999481,33441.700935,811.396677,1235.721703
D,9957.185347,11.766535,44720.325765,978.385982,1727.541955


In [17]:
customers.groupby('location').mean()

,id,number_of_purchases,value_of_purchases,total_standard_points,total_points_redeemed
location,,,,,
LA,10032.143159,9.700781,37515.460156,851.433594,1365.234375
NYC,9990.620018,12.230054,32980.989624,779.699463,1173.524150
SF,9970.998167,8.564003,15895.750430,535.764605,446.735395


In [18]:
c2 = customers.copy()

In [19]:
c2['age_'] = c2['age'].apply(lambda x: {
    'wise': 'wise', 'youthful': 'youthful', 'oldish': 'oldish & young', 'young': 'oldish & young'
}.get(x))
c2['gender_'] = c2['gender'].apply(lambda x: {
    'A': 'A', 'B': 'B, C, & D', 'C': 'B, C, & D', 'D': 'B, C, & D'
}.get(x))
c2['location_'] = c2['location'].apply(lambda x: {
    'NYC': 'NYC', 'LA': 'LA & SF', 'SF': 'LA & SF'
}.get(x))

In [20]:
le, X_train, X_test, y_train, y_test = fit_transform_split_data(
    ['favorite_movie_line', 'gender_', 'age_', 'number_of_purchases', 
     'value_of_purchases', 'total_points_redeemed', 'total_standard_points'], 
    'location_',
    c2)

973


In [21]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=5, random_state=1).fit(X_train, y_train)
y_pred = rfc.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.58      0.63      0.60       145
          1       0.38      0.33      0.35        99

avg / total       0.50      0.51      0.50       244



In [22]:
from sklearn import svm

svm_model = svm.SVC(decision_function_shape='ovo', random_state=1).fit(X_train, y_train)
y_pred = svm_model.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.60      1.00      0.75       145
          1       1.00      0.02      0.04        99

avg / total       0.76      0.60      0.46       244



In [23]:
lin_svm_model = svm.LinearSVC(random_state=1).fit(X_train, y_train)
y_pred = lin_svm_model.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

          0       0.59      0.97      0.73       145
          1       0.17      0.01      0.02        99

avg / total       0.42      0.58      0.44       244

